In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [2]:
url = "/content/combined_data.csv"
general_data = pd.read_csv(url)

In [3]:
X = general_data[['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']]
y = general_data['activity']

In [4]:
def create_time_domain_features(segment):
    features = {}
    features['mean'] = np.mean(segment)
    features['std'] = np.std(segment)
    features['max'] = np.max(segment)
    features['min'] = np.min(segment)
    return features

In [5]:
def extract_features(data, window_size=208, step_size=4):
    segments = []
    labels = []
    for start in range(0, len(data) - window_size, step_size):
        end = start + window_size
        segment = data.iloc[start:end]
        label = data['activity'].iloc[start:end].mode()[0]
        features = {}
        for column in ['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']:
            column_features = create_time_domain_features(segment[column])
            for key in column_features:
                features[f'{column}_{key}'] = column_features[key]
        segments.append(features)
        labels.append(label)
    return pd.DataFrame(segments), labels

In [7]:
window_size = 208
step_size = 4
X, y = extract_features(general_data, window_size, step_size)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
svm_model = SVC()
svm_model.fit(X_train, y_train)
svm_predictions = svm_model.predict(X_test)

In [11]:
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

In [12]:
print("SVM Classification Report")
print(classification_report(y_test, svm_predictions))

print("Random Forest Classification Report")
print(classification_report(y_test, rf_predictions))

SVM Classification Report
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      2371
     running       1.00      1.00      1.00      7647
      stairs       1.00      1.00      1.00       366
     walking       1.00      1.00      1.00      4140

    accuracy                           1.00     14524
   macro avg       1.00      1.00      1.00     14524
weighted avg       1.00      1.00      1.00     14524

Random Forest Classification Report
              precision    recall  f1-score   support

        idle       1.00      1.00      1.00      2371
     running       1.00      1.00      1.00      7647
      stairs       1.00      1.00      1.00       366
     walking       1.00      1.00      1.00      4140

    accuracy                           1.00     14524
   macro avg       1.00      1.00      1.00     14524
weighted avg       1.00      1.00      1.00     14524

